# Submit CASSCF calculations

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# General imports.
import re
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA imports.
%load_ext aiida
%aiida
# AiiDAlab imports.
import aiidalab_widgets_base as awb
from aiida import orm, plugins

from surfaces_tools.utils import wfn

# Custom imports.
from surfaces_tools.widgets import cdxml, computational_resources, editors, inputs

In [ ]:
GaussianCasscfSeriesWorkChain = WorkflowFactory("nanotech_empa.gaussian.casscf_series")

In [ ]:
# Structure selector.
structure_selector = awb.StructureManagerWidget(
    importers=[
        awb.StructureUploadWidget(title="Import from computer"),
        awb.StructureBrowserWidget(title="AiiDA database"),
        awb.SmilesWidget(title="From SMILES"),
        cdxml.CdxmlUpload2GnrWidget(title="CDXML"),
    ],
    editors=[
        awb.BasicStructureEditor(title="Edit structure"),
    ],
    storable=True,
    node_class="StructureData",
)

display(structure_selector)

# Gaussian code.
code_input_widget = awb.ComputationalResourcesWidget(
    description="Gaussian code:", default_calc_job_plugin="gaussian"
)
# formchk code.
formchk_code_input_widget = awb.ComputationalResourcesWidget(
    description="Formchk code:", default_calc_job_plugin="gaussian.formchk"
)
# cubegen code.
cubegen_code_input_widget = awb.ComputationalResourcesWidget(
    description="Cubegen code:", default_calc_job_plugin="gaussian.cubegen"
)

resources = computational_resources.ProcessResourcesWidget()

output = ipw.Output()

In [ ]:
# Functional.
functional = ipw.Dropdown(
    value="UHF",
    options=[
        "UHF","UB3LYP",
    ],
    description="Functional:",
    style={"description_width": "initial"},
)
# Basis set.
basis = ipw.Dropdown(
    value="6-311G(d,p)",
    options=[
        "6-311G(d,p)","STO-3G",
    ],
    description="Basis set:",
    style={"description_width": "initial"},
)
# (n,m) lists.
nm_list = ipw.Text(
    value="",
    description="(n,m) lists:",
    placeholder="(2,2) (4,4)",
    style={"description_width": "initial"},
)
# start calc folder.
start_calc_folder = ipw.Text(
    value="",
    description="Start calc folder:",
    placeholder="/project/s1267/cpi/",
    style={"description_width": "initial"},
)

# Multiplicities list.
multiplicities = ipw.Text(
    value="",
    description="Multiplicities:",
    placeholder="1 3",
    style={"description_width": "initial"},
)
# start uno use natural orbitals of the start calculation.
start_uno = ipw.Checkbox(
    value=True,
    description="Start UNO",
    style={"description_width": "initial"},
)
# perform CASMP2 calculation.
casmp2 = ipw.Checkbox(
    value=False,
    description="MP2 correction",
    style={"description_width": "initial"},
)
# number of orbitals to plot.
n_orbitals = ipw.IntText(
    value=0,
    description="Plot N*occ & N*virt orbitals. N=",
    style={"description_width": "initial"},
)





In [ ]:
workflow_description = ipw.Text(
    description="Workflow description:",
    placeholder="Provide the description here.",
    style={"description_width": "initial"},
    layout={"width": "70%"},
)

In [ ]:
def string_to_tuple_list(s):
    # Use regex to find all tuples in the string, handling spaces
    tuples = re.findall(r'\(\s*(\d+)\s*,\s*(\d+)\s*\)', s)
    # Convert the list of string tuples into a list of integer tuples
    return [(int(x), int(y)) for x, y in tuples]


def prepare_calculation():
    with output:
        clear_output()
    if not structure_selector.structure_node:
        can_submit, msg = False, "Select a structure first."
    elif not code_input_widget.value:
        can_submit, msg = False, "Select Gaussian code."
    elif not formchk_code_input_widget.value:
        can_submit, msg = False, "Select Formchk code."
    elif not cubegen_code_input_widget.value:
        can_submit, msg = False, "Select Cubegen code."
    else:
        can_submit, msg = True, "Submitted CASSCF workchain"

    if not can_submit:
        with output:
            print(msg)
            return
        
    builder = GaussianCasscfSeriesWorkChain.get_builder()
    builder.metadata.label = "Gaussian_CASSCF_series" 
    builder.metadata.description = workflow_description.value
    builder.gaussian_code = orm.load_code(code_input_widget.value)
    builder.formchk_code = orm.load_code(formchk_code_input_widget.value)
    builder.cubegen_code = orm.load_code(cubegen_code_input_widget.value)
    builder.nm_list = orm.List(list=string_to_tuple_list(nm_list.value))
    builder.structure = structure_selector.structure_node
    builder.init_functional = orm.Str(functional.value)
    #builder.start_calc_folder = orm.Str(start_calc_folder.value)
    builder.basis_set = orm.Str(basis.value)
    builder.multiplicity_list = orm.List(list=[int(x) for x in multiplicities.value.split(" ")])
    builder.start_uno = orm.Bool(start_uno.value)
    builder.mp2 = orm.Bool(casmp2.value)
    builder.num_orbital_cubes = orm.Int(n_orbitals.value)
    
    builder.options = orm.Dict({
        "max_wallclock_seconds": resources.walltime_seconds,
        "max_memory_kb": resources.total_memory_gb  * 1024 * 1024,
        "resources": {
            "num_machines": resources.nodes,
            "tot_num_mpiprocs": resources.tasks_per_node,
        },
    }
    )

    return builder

In [ ]:
btn_submit = awb.SubmitButtonWidget(
    GaussianCasscfSeriesWorkChain,
    inputs_generator=prepare_calculation,
    disable_after_submit=False,
    append_output=True,
)

## Parameters

In [ ]:
display(ipw.VBox([nm_list, functional, basis, start_calc_folder, multiplicities, start_uno, casmp2, n_orbitals]))

# Code and resources

In [ ]:
display(code_input_widget,
        formchk_code_input_widget,
        cubegen_code_input_widget,
        resources,)

# Submit

In [ ]:
display(workflow_description, btn_submit, output)